In [64]:
import numpy as np
import pandas as pd

In [65]:
train_data = pd.read_csv('./mnist_train.csv')
test_data = pd.read_csv('./mnist_test.csv')

In [66]:
def train_test_split(train_data, test_data):
    X_train = train_data.drop(['label'],axis=1)
    y_train = train_data['label']
    X_test = test_data.drop(['label'],axis=1)
    y_test = test_data['label']
    return X_train, y_train, X_test, y_test

def to_numpy(X_train, y_train, X_test, y_test):
    X_train = X_train.to_numpy()
    y_train = y_train.to_numpy()
    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()
    return X_train, y_train, X_test, y_test
    

# Hebbian Neural Network

In [33]:
class Hebbian_NN():
    def __init__(self, input_size, output_size):
        self.w = np.zeros((output_size, input_size + 1))

    def fit(self, X_train, y_train):
        for i,row in enumerate(X_train):
            label = y_train[i]
            x = np.append(row,1)
            self.w[label] = self.w[label] + x #* label

    def predict(self, x):
        x = np.append(x,1)
        outputs = np.dot(self.w,x)
        return np.argmax(outputs)

    def evaluate(self, X_test, y_test):
        preds = np.array([])
        for x in X_test:
            pred = self.predict(x)
            preds = np.append(preds, pred)
            
        print(f'accuracy: {(preds == y_test).sum() / len(y_test)}')

In [34]:
X_train, y_train, X_test, y_test = train_test_split(train_data, test_data)
X_train, y_train, X_test, y_test = to_numpy(X_train, y_train, X_test, y_test)

In [35]:
heb1 = Hebbian_NN(784, 10)
heb1.fit(X_train, y_train)
heb1.evaluate(X_test,y_test)

accuracy: 0.6804


In [36]:
X_train, y_train, X_test, y_test = train_test_split(train_data, test_data)
X_train = X_train.apply(lambda col: col.map(lambda x: 1 if x > 0 else 0))
X_test = X_test.apply(lambda col: col.map(lambda x: 1 if x > 0 else 0))
X_train, y_train, X_test, y_test = to_numpy(X_train, y_train, X_test, y_test)

In [37]:
model2 = Hebbian_NN(784, 10)
model2.fit(X_train, y_train)
model2.evaluate(X_test,y_test)

accuracy: 0.6137


In [38]:
X_train, y_train, X_test, y_test = train_test_split(train_data, test_data)
X_train = X_train.apply(lambda col: col.map(lambda x: x if x > 0 else -1))
X_test = X_test.apply(lambda col: col.map(lambda x: x if x > 0 else -1))
X_train, y_train, X_test, y_test = to_numpy(X_train, y_train, X_test, y_test)

In [39]:
model3 = Hebbian_NN(784, 10)
model3.fit(X_train, y_train)
model3.evaluate(X_test,y_test)

accuracy: 0.6825


In [40]:
X_train, y_train, X_test, y_test = train_test_split(train_data, test_data)
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train, y_train, X_test, y_test = to_numpy(X_train, y_train, X_test, y_test)

In [41]:
model4 = Hebbian_NN(784, 10)
model4.fit(X_train, y_train)
model4.evaluate(X_test,y_test)

accuracy: 0.6808


In [42]:
X_train, y_train, X_test, y_test = train_test_split(train_data, test_data)
X_train = X_train - X_train.mean()
X_test = X_test - X_train.mean()
X_train, y_train, X_test, y_test = to_numpy(X_train, y_train, X_test, y_test)
model5 = Hebbian_NN(784, 10)
model5.fit(X_train, y_train)
model5.evaluate(X_test,y_test)

accuracy: 0.6451


In [43]:
X_train, y_train, X_test, y_test = train_test_split(train_data, test_data)
X_train = X_train - X_train.mean()
X_test = X_test - X_train.mean()

from sklearn.decomposition import PCA
pca = PCA(whiten=True)
X_train = pd.DataFrame(pca.fit_transform(X_train))
X_test = pd.DataFrame(pca.transform(X_test))

X_train, y_train, X_test, y_test = to_numpy(X_train, y_train, X_test, y_test)

In [44]:
model6 = Hebbian_NN(784, 10)
model6.fit(X_train, y_train)
model6.evaluate(X_test,y_test)

accuracy: 0.8096


# Perseptron Neural Network

In [45]:
X_train, y_train, X_test, y_test = train_test_split(train_data, test_data)
X_train, y_train, X_test, y_test = to_numpy(X_train, y_train, X_test, y_test)

In [46]:
class Perseptron_NN():
    def __init__(self, input_size, output_size):
        self.w = np.zeros((output_size, input_size + 1))

    def fit(self, X_train, y_train):
        for i,row in enumerate(X_train):
            label = y_train[i]
            if self.predict(row) != label:
                self.w[label] = self.w[label] + np.append(row,1) #* label

    def predict(self, x):
        x = np.append(x,1)
        outputs = np.dot(self.w,x)
        return np.argmax(outputs)

    def evaluate(self, X_test, y_test):
        preds = np.array([])
        for x in X_test:
            pred = self.predict(x)
            preds = np.append(preds, pred)
            
        print(f'accuracy: {(preds == y_test).sum() / len(y_test)}')

In [47]:
pr1 = Perseptron_NN(784, 10)
pr1.fit(X_train, y_train)
pr1.evaluate(X_test,y_test)

accuracy: 0.8668


# Adaline Neural Network

In [72]:
class Adaline:
    def __init__(self, learning_rate=0.0001, epochs=50):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features) 
        self.bias = 0 

        for epoch in range(self.epochs):
            linear_output = np.dot(X, self.weights) + self.bias
            errors = y - linear_output
            
            self.weights += self.learning_rate * np.dot(X.T, errors) / n_samples
            self.bias += self.learning_rate * np.mean(errors)
            
            loss = np.mean(errors ** 2) / 2
            print(f"Epoch {epoch + 1}/{self.epochs}, Loss: {loss:.4f}")

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return np.where(linear_output >= 0, 1, -1)


In [74]:
X_train, y_train, X_test, y_test = train_test_split(train_data, test_data)
X_train, y_train, X_test, y_test = to_numpy(X_train, y_train, X_test, y_test)

X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = np.where(y_train == 0, -1, 1)
y_test = np.where(y_test == 0, -1, 1)

In [75]:
adaline = Adaline(learning_rate=0.0001, epochs=50)
adaline.fit(X_train, y_train)
y_pred = adaline.predict(X_test)

accuracy = np.mean(y_pred == y_test) * 100
print(f"Accuracy: {accuracy:.2f}%")

Epoch 1/50, Loss: 0.5000
Epoch 2/50, Loss: 0.4978
Epoch 3/50, Loss: 0.4956
Epoch 4/50, Loss: 0.4934
Epoch 5/50, Loss: 0.4912
Epoch 6/50, Loss: 0.4891
Epoch 7/50, Loss: 0.4869
Epoch 8/50, Loss: 0.4848
Epoch 9/50, Loss: 0.4827
Epoch 10/50, Loss: 0.4806
Epoch 11/50, Loss: 0.4785
Epoch 12/50, Loss: 0.4765
Epoch 13/50, Loss: 0.4744
Epoch 14/50, Loss: 0.4724
Epoch 15/50, Loss: 0.4704
Epoch 16/50, Loss: 0.4684
Epoch 17/50, Loss: 0.4664
Epoch 18/50, Loss: 0.4644
Epoch 19/50, Loss: 0.4625
Epoch 20/50, Loss: 0.4605
Epoch 21/50, Loss: 0.4586
Epoch 22/50, Loss: 0.4567
Epoch 23/50, Loss: 0.4548
Epoch 24/50, Loss: 0.4529
Epoch 25/50, Loss: 0.4510
Epoch 26/50, Loss: 0.4492
Epoch 27/50, Loss: 0.4473
Epoch 28/50, Loss: 0.4455
Epoch 29/50, Loss: 0.4437
Epoch 30/50, Loss: 0.4419
Epoch 31/50, Loss: 0.4401
Epoch 32/50, Loss: 0.4383
Epoch 33/50, Loss: 0.4365
Epoch 34/50, Loss: 0.4348
Epoch 35/50, Loss: 0.4330
Epoch 36/50, Loss: 0.4313
Epoch 37/50, Loss: 0.4296
Epoch 38/50, Loss: 0.4279
Epoch 39/50, Loss: 0.